# Notebook **Four** - Creating a fiducial redshift table from tool & literature identified spectroscopic redshifts

## Import statements and reading history/sample

In [1]:
from ident_run_setup import cosmo, side_length, HISTORY_FILE_PATH, load_history, proj_name, update_history

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 50)
import json
import os

%matplotlib inline

In [2]:
cur_history = load_history()

Made a mistake in an earlier version of the spectral identification class, and the 6dF observation spectrum ID was just called 'spec_id' in the history, which clashes with some other headers when I'm constructing the multi-index dataframe below:

In [3]:
spec_ident = str(cur_history['bcg_spec_identification'])
if "'survey_spec_id': {'spec_id'" in spec_ident:
    spec_ident = spec_ident.replace("'survey_spec_id': {'spec_id'", "'survey_spec_id': {'obs_spec_id'")
    spec_ident = eval(spec_ident)
    cur_history['bcg_spec_identification'] = spec_ident
    update_history(cur_history)

In [4]:
samp = pd.read_csv(cur_history['static_samp_file'])
# This means we can use .loc[ENTER NAME HERE] to get the information for a particular cluster
samp = samp.set_index('name')

# This method shows the top two lines of the loaded-in sample file
samp.head(2)

,LoVoCCSID,common_name,cent_im_ra,cent_im_dec,redshift,MCXC_R500,MCXC_RA,MCXC_DEC,manual_xray_ra,manual_xray_dec,MCXC_Lx500_0.1_2.4,ang_prop_ratio
name,,,,,,,,,,,,
LoVoCCS-1,1,A2029,227.7343,5.745471,0.0766,1.3344,227.73,5.72,227.7343,5.745471,8.726709e+44,86.035164
LoVoCCS-2,2,A401,44.7400,13.580000,0.0739,1.2421,44.74,13.58,NaN,NaN,6.088643e+44,83.258258


## Reading the identified BCG sample file

In [5]:
bcg_samp = pd.read_csv("outputs/bcg_output_sample.csv")
bcg_samp

,cluster_name,no_bcg_cand,BCG1_desi-ls_ra,BCG1_desi-ls_dec,BCG2_desi-ls_ra,BCG2_desi-ls_dec,BCG3_desi-ls_ra,BCG3_desi-ls_dec,BCG4_desi-ls_ra,BCG4_desi-ls_dec,BCG1_lovoccs_ra,BCG1_lovoccs_dec,BCG2_lovoccs_ra,BCG2_lovoccs_dec
0,LoVoCCS-1,False,227.733824,5.744883,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,LoVoCCS-2,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44.740836,13.582646,NaN,NaN
2,LoVoCCS-4A,False,10.460194,-9.302871,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,LoVoCCS-4B,False,10.429048,-9.439317,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,LoVoCCS-5,False,303.113338,-56.826500,302.710346,-56.673695,303.506670,-57.027568,303.49407,-57.039226,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,LoVoCCS-121,False,52.460588,-52.579591,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
63,LoVoCCS-122,False,156.994404,-6.798922,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64,LoVoCCS-123,False,193.671028,-29.227460,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65,LoVoCCS-131,False,137.649589,-10.582130,137.741475,-10.564283,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Putting the tool-identified spectroscopic redshifts into a Pandas dataframe

In [255]:
bcg_spec_rejig = [{'name': clust_name} | spec_info for clust_name, spec_info in cur_history['bcg_spec_identification'].items()]
spec_info = pd.json_normalize(bcg_spec_rejig)

rel_cols = spec_info.columns[(spec_info.columns == 'name') | spec_info.columns.str.contains(r'\.spec_id|z') | 
                              spec_info.columns.str.endswith('survey')]
spec_info = spec_info[rel_cols]
spec_info = spec_info.set_index('name')

spec_info.columns = pd.MultiIndex.from_tuples([tuple(col.split(".")) for col in spec_info.columns])
spec_info = spec_info.droplevel(1, axis=1)
spec_info.sort_index(axis=1, inplace=True)

In [256]:
spec_info

BCG1                                                     \
                 spec0                                              spec1   
               spec_id survey         z     z_err z_quality       spec_id   
name                                                                        
LoVoCCS-1    4634042.0    cfa  0.073284  0.000667       NaN  4.634043e+06   
LoVoCCS-2    4611177.0    cfa  0.074518  0.000500       NaN           NaN   
LoVoCCS-4A    231472.0   sdss  0.055359  0.000016       NaN  4.605303e+06   
LoVoCCS-4B   4605284.0    cfa  0.056326  0.000117       NaN  5.683825e+06   
LoVoCCS-5    4639181.0    cfa  0.055378  0.000147       NaN           NaN   
...                ...    ...       ...       ...       ...           ...   
LoVoCCS-121  4612420.0    cfa  0.063944  0.000334       NaN           NaN   
LoVoCCS-122  4493134.0    6df  0.116297       NaN       4.0  2.842364e+15   
LoVoCCS-123        NaN    NaN       NaN       NaN       NaN           NaN   
LoVoCCS-131        NaN    NaN       NaN       NaN       NaN           NaN   
LoVoCCS-134        NaN    NaN       NaN       NaN       NaN           NaN   

                                                                           \
                                                         spec2              
               survey         z     z_err z_quality    spec_id     survey   
name                                                                        
LoVoCCS-1         cfa  0.077997  0.000334       NaN  4712886.0  hectospec   
LoVoCCS-2         NaN       NaN       NaN       NaN        NaN        NaN   
LoVoCCS-4A        cfa  0.055158  0.000147       NaN  5673213.0       fast   
LoVoCCS-4B       fast  0.056123       NaN       NaN  5686980.0       fast   
LoVoCCS-5         NaN       NaN       NaN       NaN        NaN        NaN   
...               ...       ...       ...       ...        ...        ...   
LoVoCCS-121       NaN       NaN       NaN       NaN        NaN        NaN   
LoVoCCS-122  desi_dr1  0.116045  0.000018       NaN        NaN        NaN   
LoVoCCS-123       NaN       NaN       NaN       NaN        NaN        NaN   
LoVoCCS-131       NaN       NaN       NaN       NaN        NaN        NaN   
LoVoCCS-134       NaN       NaN       NaN       NaN        NaN        NaN   

                                                                         \
                                           spec3                          
                    z z_err z_quality    spec_id survey         z z_err   
name                                                                      
LoVoCCS-1    0.073512   NaN       NaN  5678635.0   fast  0.078171   NaN   
LoVoCCS-2         NaN   NaN       NaN        NaN    NaN       NaN   NaN   
LoVoCCS-4A   0.055488   NaN       NaN  5673270.0   fast  0.330414   NaN   
LoVoCCS-4B   0.056031   NaN       NaN        NaN    NaN       NaN   NaN   
LoVoCCS-5         NaN   NaN       NaN        NaN    NaN       NaN   NaN   
...               ...   ...       ...        ...    ...       ...   ...   
LoVoCCS-121       NaN   NaN       NaN        NaN    NaN       NaN   NaN   
LoVoCCS-122       NaN   NaN       NaN        NaN    NaN       NaN   NaN   
LoVoCCS-123       NaN   NaN       NaN        NaN    NaN       NaN   NaN   
LoVoCCS-131       NaN   NaN       NaN        NaN    NaN       NaN   NaN   
LoVoCCS-134       NaN   NaN       NaN        NaN    NaN       NaN   NaN   

                                                                              \
                           spec4                                       spec5   
            z_quality    spec_id survey         z z_err z_quality    spec_id   
name                                                                           
LoVoCCS-1         NaN        NaN    NaN       NaN   NaN       NaN        NaN   
LoVoCCS-2         NaN        NaN    NaN       NaN   NaN       NaN        NaN   
LoVoCCS-4A        NaN  5673330.0   fast  0.055128   NaN       NaN  5673367.0   
LoVoCCS-4B        Na

In [268]:
# spec_info.xs('spec_id', axis=1, level=-1) = 

spec_info.xs('spec_id', axis=1, level=-1).astype('Int64')


BCG1                                                        \
               spec0             spec1    spec2    spec3    spec4    spec5   
name                                                                         
LoVoCCS-1    4634042           4634043  4712886  5678635     <NA>     <NA>   
LoVoCCS-2    4611177              <NA>     <NA>     <NA>     <NA>     <NA>   
LoVoCCS-4A    231472           4605303  5673213  5673270  5673330  5673367   
LoVoCCS-4B   4605284           5683825  5686980     <NA>     <NA>     <NA>   
LoVoCCS-5    4639181              <NA>     <NA>     <NA>     <NA>     <NA>   
...              ...               ...      ...      ...      ...      ...   
LoVoCCS-121  4612420              <NA>     <NA>     <NA>     <NA>     <NA>   
LoVoCCS-122  4493134  2842363537915904     <NA>     <NA>     <NA>     <NA>   
LoVoCCS-123     <NA>              <NA>     <NA>     <NA>     <NA>     <NA>   
LoVoCCS-131     <NA>              <NA>     <NA>     <NA>     <NA>     <NA>   
LoVoCCS-134     <NA>              <NA>     <NA>     <NA>     <NA>     <NA>   

                                                    BCG2                    \
               spec6    spec7    spec8    spec9    spec0 spec1 spec2 spec3   
name                                                                         
LoVoCCS-1       <NA>     <NA>     <NA>     <NA>     <NA>  <NA>  <NA>  <NA>   
LoVoCCS-2       <NA>     <NA>     <NA>     <NA>     <NA>  <NA>  <NA>  <NA>   
LoVoCCS-4A   5673397  5673477  5673485  5673527     <NA>  <NA>  <NA>  <NA>   
LoVoCCS-4B      <NA>     <NA>     <NA>     <NA>     <NA>  <NA>  <NA>  <NA>   
LoVoCCS-5       <NA>     <NA>     <NA>     <NA>  4639141  <NA>  <NA>  <NA>   
...              ...      ...      ...      ...      ...   ...   ...   ...   
LoVoCCS-121     <NA>     <NA>     <NA>     <NA>     <NA>  <NA>  <NA>  <NA>   
LoVoCCS-122     <NA>     <NA>     <NA>     <NA>     <NA>  <NA>  <NA>  <NA>   
LoVoCCS-123     <NA>     <NA>     <NA>     <NA>     <NA>  <NA>  <NA>  <NA>   
LoVoCCS-131     <NA>     <NA>     <NA>     <NA>  4561063  <NA>  <NA>  <NA>   
LoVoCCS-134     <NA>     <NA>     <NA>     <NA>     <NA>  <NA>  <NA>  <NA>   

                BCG3           BCG4                 
               spec0 spec1    spec0    spec1 spec2  
name                                                
LoVoCCS-1       <NA>  <NA>     <NA>     <NA>  <NA>  
LoVoCCS-2       <NA>  <NA>     <NA>     <NA>  <NA>  
LoVoCCS-4A      <NA>  <NA>     <NA>     <NA>  <NA>  
LoVoCCS-4B      <NA>  <NA>     <NA>     <NA>  <NA>  
LoVoCCS-5    4639210  <NA>  4510448  4639206  <NA>  
...              ...   ...      ...      ...   ...  
LoVoCCS-121     <NA>  <NA>     <NA>     <NA>  <NA>  
LoVoCCS-122     <NA>  <NA>     <NA>     <NA>  <NA>  
LoVoCCS-123     <NA>  <NA>     <NA>     <NA>  <NA>  
LoVoCCS-131     <NA>  <NA>     <NA>     <NA>  <NA>  
LoVoCCS-134     <NA>  <NA>     <NA>     <NA>  <NA>  

[67 rows x 19 columns]

## Selecting **one** tool-identified redshift per candidate

In [76]:
all_survey = np.unique(spec_info.loc(axis=1)[:, :, 'survey'].values.astype(str))
all_survey

array(['2df', '2dflens', '6df', 'cfa', 'desi_dr1', 'eboss', 'fast',
       'hectospec', 'lamost', 'nan', 'sdss', 'uzc', 'wigglez'],
      dtype='<U9')

In [ ]:
ranking = {'sdss': 0, 'desi_dr1': 1}

In [271]:
cur_cand = 'BCG1'

fid_z_info = []
for row_ind, row in spec_info[cur_cand].iterrows():
    new_fidz_row_dat = [row.name]
    
    # The call to 'align' makes sure that the boolean slicer dataframe returned by checking the survey isn't NaN, is mapped
    #  such that it can be passed as a slicer to the overall row and just return the spec columns with actual entries in for
    #  this particular BCG
    col_with_spec = row.align(~row.loc[:, 'survey'].isnull(), level=0)[1]
    cut_row = row[col_with_spec]

    cur_num_spec = len(cut_row.index.get_level_values(0).unique())
    if cur_num_spec == 1:
        new_fidz_row_dat += cut_row['spec0'].values.tolist()

    fid_z_info.append(new_fidz_row_dat)

fiducial_z = pd.DataFrame(fid_z_info, columns=["name", "spec_id", "survey", "z", "z_err", "z_quality"])
# Can't for the life of me figure out how to change the dtype of all the spec_id columns in the multi-index 
#  Pandas column implementation - so I will change it here
fiducial_z = fiducial_z.astype({"spec_id": 'Int64'})
fiducial_z

,name,spec_id,survey,z,z_err,z_quality
0,LoVoCCS-1,<NA>,None,NaN,NaN,NaN
1,LoVoCCS-2,4611177,cfa,0.074518,0.000500,NaN
2,LoVoCCS-4A,<NA>,None,NaN,NaN,NaN
3,LoVoCCS-4B,<NA>,None,NaN,NaN,NaN
4,LoVoCCS-5,4639181,cfa,0.055378,0.000147,NaN
...,...,...,...,...,...,...
62,LoVoCCS-121,4612420,cfa,0.063944,0.000334,NaN
63,LoVoCCS-122,<NA>,None,NaN,NaN,NaN
64,LoVoCCS-123,<NA>,None,NaN,NaN,NaN
65,LoVoCCS-131,<NA>,None,NaN,NaN,NaN
